In [1]:
from huggingface_hub import login

login()

In [2]:
!wandb login <token>

wandb: WARNING Using legacy-service, which is deprecated. If this is unintentional, you can fix it by ensuring you do not call `wandb.require('legacy-service')` and do not set the WANDB_X_REQUIRE_LEGACY_SERVICE environment variable.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


In [2]:
%%capture
!pip install transformers datasets trl huggingface_hub

In [3]:
%%capture
!pip install fastapi uvicorn pyngrok peft

In [4]:
%%capture
!pip install -U bitsandbytes
import torch

In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, BitsAndBytesConfig
from datasets import load_dataset, DatasetDict
from trl import SFTConfig, SFTTrainer
import torch
from peft import LoraConfig

In [6]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

##training

Обучение адаптера QLoRa на синтезированном (переведенном с английского) разговорном датасете.

In [ ]:
model_name = "meta-llama/Llama-3.1-8B-Instruct"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype="bfloat16",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name,
    quantization_config=bnb_config,
    attn_implementation="sdpa"
).to(device)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)

finetune_name = "alinatl/Llama-3.1-8B-Instruct-qlora-spanish"
finetune_tags = ['lora', 'chat', 'spanish']

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
dataset = load_dataset("alinatl/personachat_truecased-spanish")
dataset

DatasetDict({
    train: Dataset({
        features: ['en', 'sp'],
        num_rows: 800
    })
    valid: Dataset({
        features: ['en', 'sp'],
        num_rows: 200
    })
})

In [ ]:
data_test_valid = dataset['valid'].train_test_split(test_size=0.5, seed=42)

train_data = dataset['train']
test_data = data_test_valid['test']
valid_data = data_test_valid['train']

In [ ]:
train_data = train_data.map(lambda example:
 {"text": tokenizer.apply_chat_template(
        example['sp'],
        tokenize=False,
        add_generation_prompt=False,
    batched=True)

  }
)

test_data = test_data.map(lambda example:
 {"text": tokenizer.apply_chat_template(
        example['sp'],
        tokenize=False,
        add_generation_prompt=False,
    batched=True)

  }
)

valid_data = valid_data.map(lambda example:
 {"text": tokenizer.apply_chat_template(
        example['sp'],
        tokenize=False,
        add_generation_prompt=False,
    batched=True)

  }
)

train_data['text'][0]

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 Jul 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nEl presidente Trump aquí. ¿Cómo va mi compatriota?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nHola presidente. Estoy de descanso, trabajo en una cafetería.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nCuando quiero una taza de café, voy a comprar la tienda.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nMis padres son maestros, no tenemos tu dinero.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nCuando solía tener casinos, mis padres perdían todo su dinero apostando.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nEstoy en una banda de grunge con mi hermana, así es como ganamos dinero extra.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nHe creado millones de empleos desde enero. Necesitas conseguir uno real.<|eot_id|><|start_header_id|>assistant<|en

In [ ]:
valid_data, train_data

(Dataset({
     features: ['en', 'sp'],
     num_rows: 100
 }),
 Dataset({
     features: ['en', 'sp'],
     num_rows: 800
 }))

In [ ]:
!nvidia-smi

Tue May  6 16:50:48 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   61C    P0             28W /   70W |    7348MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
rank_dimension = 4
lora_alpha = 8
lora_dropout = 0.07

per_device_train_batch_size = 1
per_device_eval_batch_size = 1
eval_accumulation_steps=4
gradient_accumulation_steps = 4

peft_config = LoraConfig(
    r=rank_dimension,
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    bias="none",
    target_modules='all-linear',
    task_type="CAUSAL_LM",
)

In [ ]:
int(train_data.num_rows/(per_device_train_batch_size*gradient_accumulation_steps)*0.1)

20

In [ ]:
args = SFTConfig(
    output_dir=finetune_name,
    num_train_epochs=1,
    per_device_train_batch_size=per_device_train_batch_size,
    per_device_eval_batch_size=per_device_eval_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    gradient_checkpointing=True,
    optim="adamw_bnb_8bit",
    learning_rate=2e-4,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="cosine",
    logging_steps=10,
    save_strategy="steps",
    save_steps=int(train_data.num_rows/(per_device_train_batch_size*gradient_accumulation_steps)*0.1),
    push_to_hub=True,
    hub_model_id=finetune_name,
    hub_strategy="every_save",
    report_to="wandb",
    max_seq_length=1512,
    packing=False,
    eval_strategy='steps',
    eval_steps=int(train_data.num_rows/(per_device_train_batch_size*gradient_accumulation_steps)*0.1),
    torch_empty_cache_steps=4,
    dataset_kwargs={
      "add_special_tokens": False,
      "append_concat_token": False,
    },
)

In [ ]:
trainer = SFTTrainer(
    model=model,
    args=args,
    train_dataset=train_data,
    eval_dataset=valid_data,
    peft_config=peft_config,
)

Converting train dataset to ChatML:   0%|          | 0/800 [00:00<?, ? examples/s]

Adding EOS to train dataset:   0%|          | 0/800 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/800 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/800 [00:00<?, ? examples/s]

Converting eval dataset to ChatML:   0%|          | 0/100 [00:00<?, ? examples/s]

Adding EOS to eval dataset:   0%|          | 0/100 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/100 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/100 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
!nvidia-smi

Tue May  6 16:51:21 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   62C    P0             28W /   70W |   11396MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
results = trainer.train()

trainer.save_model()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: alinatl to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss,Validation Loss
20,1.905500,1.707138
40,1.624300,1.593902
60,1.570800,1.546016
80,1.546000,1.522880
100,1.569700,1.513695
120,1.555100,1.507926
140,1.447400,1.500965
160,1.533600,1.495855
180,1.472100,1.493449
200,1.503500,1.493121


wandb: WARNING The get_url method is deprecated and will be removed in a future release. Please use `run.url` instead.


In [ ]:
trainer.push_to_hub(finetune_name, tags=finetune_tags)

CommitInfo(commit_url='https://huggingface.co/alinatl/Llama-3.2-3B-Instruct-qlora/commit/282c9bd0aef0635c9983a0d3bee587048e209c27', commit_message='alinatl/Llama-3.2-3B-Instruct-qlora', commit_description='', oid='282c9bd0aef0635c9983a0d3bee587048e209c27', pr_url=None, repo_url=RepoUrl('https://huggingface.co/alinatl/Llama-3.2-3B-Instruct-qlora', endpoint='https://huggingface.co', repo_type='model', repo_id='alinatl/Llama-3.2-3B-Instruct-qlora'), pr_revision=None, pr_num=None)

##inference

Инференс моей модели на первых 10-ти диалогах из тестового датасета. Сравниваю результат с базовой Лламой (без адаптера)

In [7]:
from peft import PeftModel, PeftConfig

config = PeftConfig.from_pretrained('alinatl/Llama-3.1-8B-Instruct-qlora-spanish')

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype="bfloat16",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    quantization_config=bnb_config,
    return_dict=True,
).eval().to(device)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
lora_model = PeftModel.from_pretrained(model, 'alinatl/Llama-3.1-8B-Instruct-qlora-spanish').eval().to(device)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
dataset = load_dataset("alinatl/personachat_truecased-spanish")

data_test_valid = dataset['valid'].train_test_split(test_size=0.5, seed=42)

dataset['test'] = data_test_valid['test']

In [ ]:
dataset['test']['sp'][1][:3]

[{'content': 'Hola soy derrick escuchando jazz como es U.', 'role': 'user'},
 {'content': 'Hola. ¿Cómo estás hoy?', 'role': 'assistant'},
 {'content': 'Estoy bien tratando de relajarme de una sesión de fotos.',
  'role': 'user'}]

In [ ]:
for example in dataset['test']['sp'][:10]:

  for turn in example[:4]:
      role = "👤 User" if turn['role'] == 'user' else "🤖 Assistant"
      print(f"{role}: {turn['content']}\n")

  formatted_example = tokenizer.apply_chat_template(example[:4], tokenize=True, return_dict=True, add_generation_prompt=True, return_tensors="pt").to(device)

  input_len = formatted_example['input_ids'].shape[-1]

  with torch.no_grad():
      outputs_lora = lora_model.generate(**formatted_example, max_new_tokens=200, pad_token_id=tokenizer.eos_token_id)
      outputs_base = model.generate(**formatted_example, max_new_tokens=200, pad_token_id=tokenizer.eos_token_id)

  decoded_outputs_lora = tokenizer.decode(outputs_lora[0, input_len:], skip_special_tokens=True)
  decoded_outputs_base = tokenizer.decode(outputs_base[0, input_len:], skip_special_tokens=True)
  print('qlora:', decoded_outputs_lora)
  print('base:', decoded_outputs_base)
  print()
  print('--------------------')
  print()

👤 User: ¿Cómo estás hoy?

🤖 Assistant: Estoy bien. ¡Y Dios es bueno! ¿Rezas?

👤 User: Rezo especialmente antes de mi atlántico.

🤖 Assistant: Rezo todas las mañanas para estar a salvo. ¡Tengo miedo de muchas cosas. ¡Como arañas!

qlora: Me gusta el clásico, es mi favorito. ¿Te gusta?
base: Las arañas son las mejores. ¿Tienes algún animal?

--------------------

👤 User: Hola soy derrick escuchando jazz como es U.

🤖 Assistant: Hola. ¿Cómo estás hoy?

👤 User: Estoy bien tratando de relajarme de una sesión de fotos.

🤖 Assistant: Me voy a casar muy pronto.

qlora: ¡Eso es genial! ¿Te gusta el jazz?
base: ¿Cuáles son tus hobbies?

--------------------

👤 User: ¡Espero que tu día sea agradable!

🤖 Assistant: Hola, está yendo genial. ¿Almorzaste ahora, has comido?

👤 User: Tengo un poco de pescado fresco, pero no me gusta comerlos.

🤖 Assistant: Me encantan los pescados, las verduras, no tanto.

qlora: ¿Tienes algún hobby o juego favorito?
base: ¿Qué haces para ganarte la vida?

------------

##FAST API

Супер базовый FAst Api для инференса модели. Нет поддержки контекста диалога поскольку для этого нужно продумать формат хранения контекста (бд), я решила это не реализовывать, если в задании напрямую не требуется.

Реализую в колабе (а не на локальном компе), т.к. комп без видеокарты не потянет модель в 8B парамметров.

In [8]:
from fastapi import FastAPI
from pydantic import BaseModel

from pyngrok import ngrok
import uvicorn
import threading

In [28]:
!ngrok config add-authtoken <token>

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
# (Запустить, если еще не запускали аналогичную ячейку в блоке инференс)

from peft import PeftModel, PeftConfig

config = PeftConfig.from_pretrained('alinatl/Llama-3.1-8B-Instruct-qlora-spanish')

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype="bfloat16",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    quantization_config=bnb_config,
    return_dict=True,
).eval().to(device)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
lora_model = PeftModel.from_pretrained(model, 'alinatl/Llama-3.1-8B-Instruct-qlora-spanish').eval().to(device)

In [29]:
app = FastAPI()

class Prompt(BaseModel):
    role: str
    content: str

@app.get("/")
def root():
    return {"message": 'Hola amigo, ¿cómo estás?'}


@app.post("/generate")
def generate(prompt: Prompt):
  example = [prompt]
  formatted_example = tokenizer.apply_chat_template(example, tokenize=True, return_dict=True, add_generation_prompt=True, return_tensors="pt").to(device)
  input_len = formatted_example['input_ids'].shape[-1]

  with torch.no_grad():
      outputs = lora_model.generate(**formatted_example, max_new_tokens=100, pad_token_id=tokenizer.eos_token_id)
  decoded = tokenizer.decode(outputs[0, input_len:], skip_special_tokens=True)
  return {"response": decoded}



In [30]:
public_url = ngrok.connect(8000)
print("Публичный URL:", public_url)

def run():
    uvicorn.run(app, host="0.0.0.0", port=8000)

thread = threading.Thread(target=run)
thread.start()

Публичный URL: NgrokTunnel: "https://e514-34-125-230-165.ngrok-free.app" -> "http://localhost:8000"


In [32]:
import requests

url = 'https://e514-34-125-230-165.ngrok-free.app/'

res = requests.post(url+"/generate", json={'role': 'user', 'content': '¿Cuántos cigarrillos fuma tu abuela al día?'})
print(res)
print(res.json())

INFO:     34.125.230.165:0 - "POST /generate HTTP/1.1" 200 OK
<Response [200]>
{'response': 'Ninguno, ella es un poco demasiado vieja para eso.'}
